# ViewIt AI

In [1]:
import os
import openai
import pandas as pd
from utils.prompts import *
from langchain.chains.llm import LLMChain
from langchain.tools import GooglePlacesTool
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.memory import ConversationBufferMemory
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.agents import ZeroShotAgent, AgentExecutor

In [2]:
def load_data(filename) -> pd.DataFrame:
    df = pd.read_csv(f"data/{filename}")

    df['Date'] = pd.to_datetime(df['Date'])
    # .dt.date
    return df


def create_pandas_dataframe_agent(
        llm,
        df: pd.DataFrame,
        prefix: str,
        suffix: str,
        format_instructions: str,
        verbose: bool,
        **kwargs) -> AgentExecutor:
    """Construct a pandas agent from an LLM and dataframe."""

    if not isinstance(df, pd.DataFrame):
        raise ValueError(f"Expected pandas object, got {type(df)}")

    input_variables = ["df", "input", "chat_history", "agent_scratchpad"]

    # Set up memory
    memory = ConversationBufferMemory(memory_key="chat_history")

    tools = [PythonAstREPLTool(locals={"df": df}), GooglePlacesTool()]

    prompt = ZeroShotAgent.create_prompt(
        tools=tools,
        prefix=prefix,
        suffix=suffix,
        format_instructions=format_instructions,
        input_variables=input_variables
    )
    partial_prompt = prompt.partial(df=str(df.head()))

    llm_chain = LLMChain(
        llm=llm,
        prompt=partial_prompt
    )
    tool_names = [tool.name for tool in tools]

    agent = ZeroShotAgent(llm_chain=llm_chain,
                          allowed_tools=tool_names, verbose=verbose)

    return AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        verbose=verbose,
        memory=memory,
        **kwargs
    )

In [3]:
MODEL_NAME = "gpt-4"
TEMPERATURE = 0.1
df = load_data('reidin_new.csv')
api_key = os.environ["OPENAI_API_KEY"]
org = ""

llm = ChatOpenAI(temperature=TEMPERATURE,
                    model_name=MODEL_NAME,
                    openai_api_key=api_key)

# llm = OpenAI(temperature=TEMPERATURE,
#                 model_name=MODEL_NAME,
#                 openai_api_key=api_key)

# ViewIt OpenAI API key
openai.organization = org
openai.api_key = api_key

agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    prefix=REIDIN_PREFIX,
    suffix=SUFFIX,
    format_instructions=FORMAT_INSTRUCTIONS,
    verbose=True,
    handle_parsing_errors=True,
    # max_execution_time=30,
)

C:\Users\ga201\AppData\Local\Temp\ipykernel_16292\2906168871.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Date'] = pd.to_datetime(df['Date'])


In [4]:
def ask(query, token_verbose = True):
    if token_verbose:
        with get_openai_callback() as cb:
            agent.run(query)
            print(cb)
    else:
        agent.run(query)

In [5]:
ask('What is the price of the cheapest property in Dubai Marina?')



> Entering new AgentExecutor chain...
Yes, I need to use the python_repl_ast tool to filter the dataframe for properties in Dubai Marina and then find the one with the lowest price.
Action: python_repl_ast
Action Input: df[df['Location'].str.contains('Dubai Marina')].nsmallest(1, 'Price')
Observation:       Sales Type       Date                            Location Property Type  \
34870      Ready 2022-04-10  Marina View Towers B, Dubai Marina     Apartment   

       Bedrooms  Balcony Area  Built-up Area  Plot Size   Price  \
34870         1         48.01         765.86        0.0  1275.0   

                 Developer  Studio  
34870  New City Developers   False  
Thought:I now know the final answer.
Final Answer: The cheapest property in Dubai Marina is a ready-to-move-in apartment with 1 bedroom and a built-up area of 765.86 square feet. It was sold on April 10, 2022, for a price of 1,275 AED by New City Developers.

> Finished chain.
Tokens Used: 2945
	Prompt Tokens: 2815
	Compl

In [ ]:
ask()